In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [2]:
data_path = 'C:/Users/maria/OneDrive/Documentos/Github/ECG-anomaly-detection/data/processed/'
df_normal = pd.read_parquet(data_path + 'ptdb_normal.parquet')
df_anomaly = pd.read_parquet(data_path + 'ptdb_abnormal.parquet')

In [3]:
df_normal['label'] = 0
df_anomaly['label'] = 1

In [ ]:
def prepare_autoencoder_data(normal_df, anomaly_df, test_size=0.5, random_state=42):

    # Separating the features and labels of normal data
    X_normal = normal_df.iloc[:, :-1]
    y_normal = normal_df['label']
    
    # Separating the features and labels of anomaly data
    X_anomaly = anomaly_df.iloc[:, :-1]
    y_anomaly = anomaly_df['label']
    
    # Splitting the normal data into training and testing sets
    X_train_normal, X_test_normal, y_train_normal, y_test_normal = train_test_split(
        X_normal, y_normal, test_size=test_size, random_state=random_state)
    
    # Combining the normal test data with the anomaly data to form the final test set
    X_test = pd.concat([X_test_normal, X_anomaly], axis=0)
    y_test = pd.concat([y_test_normal, y_anomaly], axis=0)
    
    # Training set remains only with normal data
    X_train = X_train_normal
    y_train = y_train_normal
    
    # Resetting the indices of the DataFrames
    X_train.reset_index(drop=True, inplace=True)
    y_train.reset_index(drop=True, inplace=True)
    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = prepare_autoencoder_data(df_normal, df_anomaly, test_size=0.2)

print(f'Training set size: {X_train.shape}')
print(f'Test set size: {X_test.shape}')


class detector(Model):
  def __init__(self):
    super(detector, self).__init__()
    self.encoder = tf.keras.Sequential([
                                        layers.Dense(32, activation='relu'),
                                        layers.Dense(16, activation='relu'),
                                        layers.Dense(8, activation='relu')
    ])
    self.decoder = tf.keras.Sequential([
                                        layers.Dense(16, activation='relu'),
                                        layers.Dense(32, activation='relu'),
                                        layers.Dense(140, activation='sigmoid')
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
